### Постановка задачи
Разделить набор данных на обучающие/проверочные в пропорции 80/20.

Загрузить данные и очистить значения. Построить модель линейной регрессии для **каждого часа в отдельности**, используя температуру воздуха (air_temperature), влажность (dew_temperature), атмосферное давление (sea_level_pressure), скорость ветра (wind_speed) и облачность (cloud_coverage).

Рассчитать качество построенной модели по проверочным данным.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных

In [10]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy_0 = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz")

### Объединение и фильтрация данных

In [11]:
# объединяем в таблицу, с помощью параметра how="left" мы указали, что главным #является датафрейм слева, название столбца – «building_id».
energy_0 = pd.merge(left=energy_0, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
# добавим отдельную серию «час» из «времени»
# проиндексируем в таблице данных energy_0 по столбцам "timestamp", "site_id"
energy_0.set_index(["timestamp", "site_id"], inplace=True)
# проиндексируем в таблице данных weather по столбцам "timestamp", "site_id"
weather.set_index(["timestamp", "site_id"], inplace=True)
# объединяем в таблицу: слева данные energy_0, справа данные weather
energy_0 = pd.merge(left=energy_0, right=weather, how="left",
                   left_index=True, right_index=True)
# сбросим индекс по данным  energy_0
energy_0.reset_index(inplace=True)
# отфильтруем только положительные значения по столбцу meter_reading
energy_0 = energy_0[energy_0["meter_reading"] > 0]
# преобразовываем формат времени в столбце timestamp
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
# атрибут dt.hour возвращает массив данных  timestamp к базовым данным (2,4…) - # добавим отдельную серию «час» из «времени»
energy_0["hour"] = energy_0["timestamp"].dt.hour
# просмотрим первые строки
print (energy_0.head())

               timestamp  site_id  building_id  meter  meter_reading  \
704  2016-01-30 08:00:00        0            0      0        43.6839   
725  2016-01-31 05:00:00        0            0      0        37.5408   
737  2016-01-31 17:00:00        0            0      0        52.5571   
2366 2016-04-08 14:00:00        0            0      0        59.3827   
2923 2016-05-01 19:00:00        0            0      0       448.0000   

     primary_use  square_feet  year_built  floor_count  air_temperature  \
704    Education         7432      2008.0          NaN              8.3   
725    Education         7432      2008.0          NaN             12.8   
737    Education         7432      2008.0          NaN             20.6   
2366   Education         7432      2008.0          NaN             21.7   
2923   Education         7432      2008.0          NaN             31.1   

      cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
704              NaN              6.

### Очистка данных

In [13]:
#очистим данные: 
# для температуры воздуха, влажности, ветра и облачности заменим отсутствующие значения 0
energy_0["air_temperature"].fillna(0, inplace=True)
energy_0["cloud_coverage"].fillna(0, inplace=True)
energy_0["dew_temperature"].fillna(0, inplace=True)
energy_0["wind_speed"].fillna(0, inplace=True)
#для осадков дополнительно занулим отрицательные значения
energy_0["precip_depth_1_hr"] = energy_0["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
# для давления заменим отсутствующие значения средними
energy_0_sea_level_pressure_mean = energy_0["sea_level_pressure"].mean()
energy_0["sea_level_pressure"] = energy_0["sea_level_pressure"].apply(lambda x:energy_0_sea_level_pressure_mean if x!=x else x)
# для ветра заменим отсутствующие значения средними
energy_0_wind_direction_mean = energy_0["wind_direction"].mean()
energy_0["wind_direction"] = energy_0["wind_direction"].apply(lambda x:energy_0_wind_direction_mean if x!=x else x)
print (energy_0.info())
#Посмотрим сводную информацию методом info():
#мы узнаем, что в таблице 5411 строки (от 704 до 8783), 
# 17 столбцов - количество ненулевых значений (non-null), тип данных: 
#  int64 - целые числа (числа размером 8 байт);
# object - абстракция Python для данных,
# float64 – число с плавающей запятой (числа размером 8 байт).
# datetime64 - выполнять операции, связанные с датой

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5411 entries, 704 to 8783
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   timestamp           5411 non-null   datetime64[ns]
 1   site_id             5411 non-null   int64         
 2   building_id         5411 non-null   int64         
 3   meter               5411 non-null   int64         
 4   meter_reading       5411 non-null   float64       
 5   primary_use         5411 non-null   object        
 6   square_feet         5411 non-null   int64         
 7   year_built          5411 non-null   float64       
 8   floor_count         0 non-null      float64       
 9   air_temperature     5411 non-null   float64       
 10  cloud_coverage      5411 non-null   float64       
 11  dew_temperature     5411 non-null   float64       
 12  precip_depth_1_hr   5411 non-null   float64       
 13  sea_level_pressure  5411 non-null   float64   

### Разделение данных

In [14]:
# Данные подготовлены для расчетов
# Выделим 20% всех данных на проверку, остальные оставим на обучение
# используя модуль train_test_split разделяем данные для машинного обучения.
# переменная test_size фактически указывает пропорцию набора тестов – 20%.
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())

               timestamp  site_id  building_id  meter  meter_reading  \
4700 2016-07-14 20:00:00        0            0      0        252.547   
8011 2016-11-29 19:00:00        0            0      0        182.926   
3988 2016-06-15 04:00:00        0            0      0        319.438   
7885 2016-11-24 13:00:00        0            0      0        177.466   
5910 2016-09-03 06:00:00        0            0      0        249.135   

     primary_use  square_feet  year_built  floor_count  air_temperature  \
4700   Education         7432      2008.0          NaN             32.2   
8011   Education         7432      2008.0          NaN             28.9   
3988   Education         7432      2008.0          NaN             25.0   
7885   Education         7432      2008.0          NaN             18.3   
5910   Education         7432      2008.0          NaN             24.4   

      cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
4700             0.0             22.

### Линейная регрессия по часам
Модель включает air_temperature, dew_temperature, sea_level_pressure, wind_speed, cloud_coverage

In [16]:
#Модель включает: 
# температуру воздуха (air_temperature), 
# влажность (dew_temperature), 
# атмосферное давление (sea_level_pressure), 
# скорость ветра (wind_speed), 
# облачность (cloud_coverage),
# а так же топливный эквивалент (meter_reading)

hours = range(0,24)
energy_0_train_lr = pd.DataFrame(energy_0_train,
    columns=["meter_reading", "air_temperature", "dew_temperature",
             "sea_level_pressure", "wind_speed", "cloud_coverage", "hour"])
# заведем списки параметров модели, добавим в него все коэффициенты для каждого часа

energy_0_lr = [[]]*len(hours)
for hour in hours:
    energy_0_train_lr_hourly = energy_0_train_lr[energy_0_train_lr["hour"]==hour]
    y = energy_0_train_lr_hourly["meter_reading"]
    x = energy_0_train_lr_hourly.drop(labels=["meter_reading","hour"], axis=1)
    model = LinearRegression().fit(x, y)
    energy_0_lr[hour] = model.coef_
    # с помощью функции append объединим два массива 
    energy_0_lr[hour] = np.append(energy_0_lr[hour], model.intercept_)
    del energy_0_train_lr_hourly
print (energy_0_lr)

[array([ 5.44405199e+00,  1.61424619e+00, -5.42134894e-01, -3.04074328e+00,
       -6.54427991e-01,  6.36035032e+02]), array([ 6.62433690e+00,  4.40658626e-01, -1.22318773e+00, -2.97220626e+00,
       -1.04520925e+00,  1.32323999e+03]), array([   7.09177915,    1.05324252,    0.36015524,   -1.5691924 ,
         -1.4446757 , -310.42659221]), array([ 7.10499829e+00,  1.02204451e-01, -5.71783693e-02, -2.93316073e+00,
        3.83580096e+00,  1.34698975e+02]), array([ 8.70225723e+00, -1.77968944e+00, -1.07036592e+00, -4.18565821e+00,
        3.11944288e+00,  1.17247560e+03]), array([   9.85191663,   -2.38297073,    0.39767277,   -1.30570005,
         -0.50021534, -339.2809219 ]), array([ 8.16563043e+00, -7.44710389e-01, -5.28029943e-02, -1.80253887e+00,
       -1.15038810e+00,  1.32915948e+02]), array([  7.23692888,   0.49470246,   0.15269225,  -1.20293475,
         1.67882484, -84.15323242]), array([ 1.10609184e+01, -3.78086359e+00, -6.92343330e-01, -4.63356425e+00,
        2.24392908e+00

### Предсказание и оценка модели

In [7]:
def calculate_model (x):
    model = energy_0_lr[x.hour]
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_lr = np.log(1 + x.air_temperature * model[0] + 
        x.dew_temperature * model[1] + x.sea_level_pressure * model[2] +
        x.wind_speed * model[3] + x.cloud_coverage * model[4] + model[5])
    x["meter_reading_lr_q"] = (meter_reading_log - meter_reading_lr)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_0_test_lr_rmsle = np.sqrt(energy_0_test["meter_reading_lr_q"].sum() / len(energy_0_test))
print ("Качество почасовой линейной регрессии, 5 параметров:",
       energy_0_test_lr_rmsle, round(energy_0_test_lr_rmsle, 1))

Качество почасовой линейной регрессии, 5 параметров: 0.2186534211678374 0.2
